In [142]:
al = [chr(i) for i in range(ord('А'), ord('Я') + 1) if i is not ord('Ё')]
print(al)

['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я']


In [2]:
ord('А') # порядковый номер первого символа русского алфавита в UTF-8

1040

Добавим в ноутбук вспомогательные функции из предыдущей работы:

In [3]:
import requests
from bs4 import BeautifulSoup as bs

def clear_paragraphs(ps:list, start=14, stop=16):
    return ''.join([i.text for i in ps[start:stop]]).replace('\xad', '').replace('\xa0', '')

def test_text(url='https://spacemorgue.com/matter-matters/'):
    '''
    Возвращает тестовый текст длиной 2408 символов
    '''
    soup = bs(requests.get(url).text, 'html.parser')
    return clear_paragraphs(soup.select_one('article').find_all('p'))

def prepare(text, specials={',':'ЗПТ','.':'ТЧК', ' ':'', '—':'ТИРЕ', '«':'КВЧ', '»':'КВЧ'}):
    '''
    Возвращает текст, удаляя пробельные символы в тексте и заменяя пунктуацию телеграфными обозначениями
    '''
    for i in specials.keys():
        text = text.replace(i, specials[i])
    return text

def comp(text, specials={'ЗПТ':',','ТЧК':'.','ТИРЕ':'—','КВЧ':'"'}):
    '''
    Возвращает текст, заменяя телеграфные обозначения пунктуацией
    '''
    for i in specials.keys():
        text = text.replace(i, specials[i])
    return text

### Шифр Тритемия 

$Y_j= X_{i+j-1}\mod n$, где:

- $X$ – исходный (открытый) текст
- $Y$ – зашифрованный текст $i$–порядковый номер буквы в алфавите таблицы, i=1...n
- $j$ – порядковый номер буквы в тексте, j=1...k
- $k$ – количество букв в тексте
- $n$ – количество букв в выбранном алфавите(мощность алфавита).

In [4]:
def trit(plaintext, alphabet):
    ciphertext = list() 
    for i, ch in enumerate(prepare(plaintext)):
        index = (alphabet.index(ch) + i) % len(alphabet)
        ciphertext.append(alphabet[index])
    return ''.join(ciphertext)

In [5]:
def trit_decrypt(ciphertext, alphabet):
    plaintext = list()
    for i, ch in enumerate(ciphertext):
        index = (alphabet.index(ch) - i) % len(alphabet)
        plaintext.append(alphabet[index])
    return comp(''.join(plaintext))

In [6]:
text = prepare('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.')
trit(text, al)

'НЖДФГПОРПШЬХЮЫЕЧВСЧЕЫДИЩПЛЯИДЕПЗЛФЙРДКШЩЯУ'

In [7]:
trit_decrypt('НЖДФГПОРПШЬХЮЫЕЧВСЧЕЫДИЩПЛЯИДЕПЗЛФЙРДКШЩЯУ', al)

'НЕВСЯКИЙ,КТОЧИТАЕТ,ВЧТЕНИИСИЛУЗНАЕТ.'

In [8]:
text = test_text().upper().replace('-', '—')
al.extend(['X', 'I'])
f'{trit(text,al)[:50]}...'

'НЖНЛСКПФИЖТЬЮНXЧIБЪИЩДXЧХЖИБСГЙКМРТЭУРТЗЖЦШСМШЪЮЛI...'

In [9]:
f'{trit_decrypt(trit(text,al),al)[:50]}...'

'НЕЛИНЕЙНАЯИСТАТИСТИЧЕСКАЯПРИЧИННОСТЬСНОВАПРИВНОСЯТ...'

Получим таблицу Тритемия в виде словаря:

In [10]:
def num(ch, start=1040):
    return ord(ch) - start

def shift(ch, n, start=1040, l=32):
    return chr((num(ch) + n) % l + start)

def make_table(al):
    table = dict()
    for i in al:
        table[i] = dict()
        for j in al:
            table[i][j] = shift(i, num(j))
    return table
        
        
print(make_table(al)['Я']['Я'])

Ю


In [11]:
def trithemius(plaintext, al):
    table = make_table(al)
    ciphertext = list()
    for i, ch in enumerate(prepare(plaintext)): # enumerate возвращает i-номер символа в тексте и ch - сам символ
        row = al[i % len(al)] # ключом в словаре является символ, так что номер в тексте требуется преобразовать 
        ciphertext.append(table[ch][row])
    return ''.join(ciphertext)

In [12]:
text = prepare('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.')
trithemius(text, al)

'НЖДФГПОРПШЬХЮЫЕЧВСЧЕЫДИЩПЛЯИДЕПЗУМЗОВИЦЧЭС'

In [13]:
def trithemius_decrypt(ciphertext, al):
    table = make_table(al)
    plaintext = list()
    for i, ch in enumerate(ciphertext):
        row = al[(len(al) - i) % len(al)]
        plaintext.append(table[ch][row])
    return comp(''.join(plaintext))

In [14]:
trithemius_decrypt('НЖДФГПОРПШЬХЮЫЕЧВСЧЕЫДИЩПЛЯИДЕПЗЛФЙРДКШЩЯУ', al)

'НЯМУБМКЛЙСФМФРЩКФВЗФЙСФДЩФЗПККУКНХЙЙМЙЦЦЫО'

### Шифр Белазо

In [15]:
def bellasо(plaintext, al, key):
    repeats = len(plaintext) // len(key)
    end = len(plaintext) % len(key)
    ciphertext = list()
    table = make_table(al)
    keystring = key * repeats + key[:end]
    
    for i, k in zip(plaintext, keystring):
        ciphertext.append(table[k][i])
    return ''.join(ciphertext)

In [16]:
def bel(plaintext, al, key):
    repeats = len(plaintext) // len(key)
    end = len(plaintext) % len(key)
    keystring = key * repeats + key[:end]
    ciphertext = list()
    for ch, k in zip(prepare(plaintext), keystring):
        index = (al.index(ch) + al.index(k)) % len(al)
        ciphertext.append(al[index])
    return ''.join(ciphertext)

In [17]:
def bel_dec(ciphertext,al,key):
    repeats = len(ciphertext) // len(key)
    end = len(ciphertext) % len(key)
    keystring = key * repeats + key[:end]
    plaintext = list()
    for ch, k in zip(prepare(ciphertext), keystring):
        index = (al.index(ch) - al.index(k)) % len(al)
        plaintext.append(al[index])
    return comp(''.join(plaintext))

In [18]:
text = prepare('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.')
bel(text, al, 'УОРХОЛЛ')

'XУТДЛХУЬХЯЕШЭЩИЦАХУЭТАXТКXРШЫЦIЭЩЮТXОХЕXАХ'

In [19]:
bel_dec(bel(text, al, 'УОРХОЛЛ'),al,'УОРХОЛЛ')

'НЕВСЯКИЙ,КТОЧИТАЕТ,ВЧТЕНИИСИЛУЗНАЕТ.'

In [21]:
f'{bel(text,al,"УОРХОЛЛ")[:50]}...'

'XУТДЛХУЬХЯЕШЭЩИЦАХУЭТАXТКXРШЫЦIЭЩЮТXОХЕXАХ...'

In [22]:
f'{bel_dec(bel(text,al,"УОРХОЛЛ"), al,"УОРХОЛЛ")[:50]}'

'НЕВСЯКИЙ,КТОЧИТАЕТ,ВЧТЕНИИСИЛУЗНАЕТ.'

In [23]:
text = prepare('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.')
bellasо(text, al, 'УОРХОЛЛ')

'АУТЖНХУЬХЯЗШЭЩКЦВХУЭТВАТМАРШЫЦБЭЩЮТАОХЗАВХ'

In [24]:
def bellaso_decrypt(ciphertext, al, key):
    repeats = len(ciphertext) // len(key)
    end = len(ciphertext) % len(key)
    plaintext = list()
    table = make_table(al)
    keystring = key * repeats + key[:end]
    
    for i, k in zip(ciphertext, keystring):
        for j in range(len(table[i])):
            ch = table[k][al[j]]
            if table[k][ch] == i:
                plaintext.append(ch)
                break
    return comp(''.join(plaintext))

In [25]:
bellaso_decrypt('АУТЖНХУЬХЯЗШЭЩКЦВХУЭТВАТМАРШЫЦБЭЩЮТАОХЗАВХ', al, 'УОРХОЛЛ')

'НЕВСЯКИЙ,КТОЧИТАЕТ,ВЧТЕНИИСИЛУЗНАЕТ.'

### Шифр Виженера с самоключом

Имплементация без специальной структуры данных:

In [92]:
def vigenere(text, key, al):
    ciphertext = list()
    # zip возвращает кортеж из массивов 1) сивмолов открытого текста 
    # 2) символов открытого текста, начинающегося с ключа
    for i,j in zip(list(text), [i for i in (key + text)[:-len(key)]]):
        index = (al.index(i) + al.index(j)) % len(al)
        ciphertext.append(al[index])
    return ''.join(ciphertext)

In [93]:
text = prepare('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.')
vigenere(text, 'Э', al)

'КТЗУРЙТСРЦБЬЬАЕЯЪТЕЧЩЦБФЩЙЧТХРЩЩУЮЪФНЕЧДЙБ'

In [97]:
def vigenere_decrypt(ciphertext, key, al):
    plaintext = list()
    keystring = [key]
    for i, ch in enumerate(ciphertext):
        index = (al.index(ch) - al.index(keystring[i])) % len(al)
        plaintext.append(al[index])
        keystring.append(al[index])
    return comp(''.join(plaintext))

In [98]:
vigenere_decrypt('КТЗУРЙТСРЦБЬЬАЕЯЪТЕЧЩЦБФЩЙЧТХРЩЩУЮЪФНЕЧДЙБ', 'Э', al)

'НЕВСЯКИЙ,КТОЧИТАЕТ,ВЧТЕНИИСИЛУЗНАЕТ.'

In [144]:
text = prepare(test_text().upper().replace('-', '—'))
ext = al.copy()
ext.extend(['X', 'I'])

In [148]:
f'{vigenere(text,"Э", ext)[:150]}...'

'ИТРУХТОЦНЯЕЩБТТЪЩБЪЯЬЦЫКЯМЯШЯЯХЪЫЯБМЛЮЫРВПЯШКПЫЯОПФЦЬУЩЯЯАГЫЭПЖШЫБЫФГЮДЖКУЪЗЛЕЗЫЧЪУIДПЩЬЩФУЫЯБМБЦIЬШXXЮРЯXЬДЛПЭXЧХНКНСXЭЛIНРТХНСЫЭЫКПЪЕМЭЯБXЛКЪЫЧУШЫЕЦ...'

In [151]:
f'{vigenere_decrypt(vigenere(text,"Э", ext), "Э", ext)[:150]}...'

'НЕЛИНЕЙНАЯИСТАТИСТИЧЕСКАЯПРИЧИННОСТЬСНОВАПРИВНОСЯТВФИЛОСОФСКУЮКОНЦЕПЦИЮКАУЗАЛЬНОЙСВЯЗИСЛОЖНОСТЬ,КОТОРАЯБЫЛАПОТЕРЯНАСПОЯВЛЕНИЕМПОНЯТИЯПОСТОЯННОЙКОНЪЮНК...'

Имплементация со словарем-таблицей:

In [30]:
def vigenere(text, key, table):
    ciphertext = list()
    # zip возвращает кортеж из массивов 1) сивмолов открытого текста 
    # 2) символов открытого текста, начинающегося с ключа
    for i,j in zip(list(text), [i for i in (key + text)[:-len(key)]]):
        ciphertext.append(table[i][j])
    return ''.join(ciphertext)

In [31]:
def vigenere_decrypt(ciphertext, key, table):
    plaintext = list()
    keystring = [key]
    for i in ciphertext:
        for j in range(len(table[i])):
            ch = table[keystring[-1]][al[j]]
            if table[keystring[-1]][ch] == i:
                plaintext.append(ch)
                keystring.append(ch)
                break
    return comp(''.join(plaintext))

In [32]:
table = make_table(al)
text = prepare('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.')
text, '->', vigenere(text, 'Э', table)

('НЕВСЯКИЙЗПТКТОЧИТАЕТЗПТВЧТЕНИИСИЛУЗНАЕТТЧК',
 '->',
 'КТЗУРЙТСРЦБЬЬАЕЯЪТЕЧЩЦБФЩЙЧТХРЩЩУЮЪФНЕЧДЙБ')

In [33]:
table['К']['Ж']

'Р'

Расшифровка чужой карточки:

In [34]:
ciphertext = 'ОУЮЪБАИПОЦНЪВШУДСЯББТЦЫЖЖШФУЬТАТЙУ'
plaintext = list()
keystring = ['Б'] + list(ciphertext)
for i in ciphertext:
    key_ch = keystring.pop(0)
    for j in range(len(table[i])):
        ch = table[key_ch][al[j]]
        if table[key_ch][ch] == i:
            plaintext.append(ch)
            break

print(comp(''.join(plaintext)))

НЕЛЬЗЯИЗЯИЧНИЦЫСНОВАСДЕЛАТЬЯЙЦО.


### Шифр Виженера с ключом-шифртекстом

In [165]:
def tabula_recta(al, x, y):
    return al[(al.index(x) + al.index(y)) % len(al)]

def tabula_recta_rev(al, x, y):
    return al[(al.index(x) - al.index(y)) % len(al)]

In [110]:
def vigenere_feedback(al, plaintext, s):
    if len(plaintext) == 1:
        return tabula_recta(al, plaintext.pop(), s)
    crypt = tabula_recta(al, plaintext.pop(0), s)
    return crypt + vigenere_feedback(al, plaintext, crypt)

In [123]:
text = prepare('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.')
text, '->', vigenere_feedback(al,list(text), 'Н')

('НЕВСЯКИЙЗПТКТОЧИТАЕТЗПТВЧТЕНИИСИЛУЗНАЕТТЧК',
 '->',
 'ЪЯIАЯIIXIXАКЬXЕНЯЯXАЗЦXXЕЧЬIIIАИУXIXXIБУXX')

In [162]:
def vigenere_feedback_dec(al, ciphertext, s):
    if len(ciphertext) == 1:
        return tabula_recta_rev(al, ciphertext.pop(), s)
    snew = ciphertext.pop(0)
    return tabula_recta_rev(al, snew, s) + vigenere_feedback_dec(al, ciphertext, snew)

In [128]:
ciphertext = vigenere_feedback(al,list(text), 'Н')
ciphertext, '->', comp(vigenere_feedback_dec(al,list(ciphertext), 'Н'))

('ЪЯIАЯIIXIXАКЬXЕНЯЯXАЗЦXXЕЧЬIIIАИУXIXXIБУXX',
 '->',
 'НЕВСЯВАIБIТКТДЧИТАБТЗПКАЧТЕЕААСИЛНБIАБТТНА')

In [153]:
text = prepare(test_text().upper().replace('-', '—'))

In [154]:
f'{vigenere_feedback(ext, list(text),"Н")[:150]}...'

'ЪЯИРЭАЙЦЦУЫКЬЬМФГХЭТЧЖРРНЬКТЗПЬЗХДЦРIМЪЬЬЙЩIБОЬЛИЪЬОЦIНЮКЮНЧИДОЬЗЭАПГЛЗССВЙЙФОЫЗРIБXЕНЮЗХЫЖФГХПЦГХЯЛЭЙЩЩЦЧРЫЫИЦЖЛЫШГГФБПМОЩЮЙСЦАПЭИЕЧЯЬЙЧЖШДБОЫЗРЪЖУЛЗ...'

In [168]:
f'{comp(vigenere_feedback_dec(ext, list(vigenere_feedback(ext, list(text),"Н")),"Н"))[:150]}...'

'НЕЛИНЕЙНАЯИСТАТИСТИЧЕСКАЯПРИЧИННОСТЬСНОВАПРИВНОСЯТВФИЛОСОФСКУЮКОНЦЕПЦИЮКАУЗАЛЬНОЙСВЯЗИСЛОЖНОСТЬ,КОТОРАЯБЫЛАПОТЕРЯНАСПОЯВЛЕНИЕМПОНЯТИЯПОСТОЯННОЙКОНЪЮНК...'

### S-блок замены ГОСТ Р 34.12-2015 [(«МАГМА»)](https://gist.github.com/empty-jack/2a6ddbe0d6852d74d16f617c11eda847/)

[RFC7836](https://datatracker.ietf.org/doc/html/rfc7836)

`id-tc26-gost-28147-param-Z`

Расшифрование выполняется так же, как и зашифрование, но инвертируется порядок подключей $K_i$.

In [130]:
from bitarray import bitarray

def splitlst(lst,n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

def cyclic_shift(a:int, b : int):
    return ((a << b) | (a >> (32 - b))) & 0xFFFFFFFF

In [134]:
def magma_sub(block : bitarray):
    assert len(block) == 32, 'Incorrect block size'
    S_box = (
    (0xc, 0x4, 0x6, 0x2, 0xa, 0x5, 0xb, 0x9, 0xe, 0x8, 0xd, 0x7, 0x0, 0x3, 0xf, 0x1),
    (0x6, 0x8, 0x2, 0x3, 0x9, 0xa, 0x5, 0xc, 0x1, 0xe, 0x4, 0x7, 0xb, 0xd, 0x0, 0xf),
    (0xb, 0x3, 0x5, 0x8, 0x2, 0xf, 0xa, 0xd, 0xe, 0x1, 0x7, 0x4, 0xc, 0x9, 0x6, 0x0),
    (0xc, 0x8, 0x2, 0x1, 0xd, 0x4, 0xf, 0x6, 0x7, 0x0, 0xa, 0x5, 0x3, 0xe, 0x9, 0xb),
    (0x7, 0xf, 0x5, 0xa, 0x8, 0x1, 0x6, 0xd, 0x0, 0x9, 0x3, 0xe, 0xb, 0x4, 0x2, 0xc),
    (0x5, 0xd, 0xf, 0x6, 0x9, 0x2, 0xc, 0xa, 0xb, 0x7, 0x8, 0x1, 0x4, 0x3, 0xe, 0x0),
    (0x8, 0xe, 0x2, 0x5, 0x6, 0x9, 0x1, 0xc, 0xf, 0x4, 0xb, 0x0, 0xd, 0xa, 0x3, 0x7),
    (0x1, 0x7, 0xe, 0xd, 0x0, 0x5, 0x8, 0x3, 0x4, 0xf, 0xa, 0x6, 0x9, 0xc, 0xb, 0x2),
    )
    newblock = ''
    for i, chunk in enumerate(splitlst(block, 4)):
        newchunk = S_box[i][int(chunk.to01(), 2)]
        newblock += f'{newchunk:04b}'
    return bitarray(f'{cyclic_shift(int(newblock, 2), 11):b}')

In [132]:
def strtobitarray(text, encode='utf-8'):
    bits = bitarray()
    bits.frombytes(text.encode('utf-8'))
    return bits

In [133]:
magma_sub(strtobitarray('test'))

1001
10011001
100110011010
1001100110100100
10011001101001001101
100110011010010011010110
1001100110100100110101101100
10011001101001001101011011000000


bitarray('100110101101100000010011001101')

In [135]:
for block in splitlst(strtobitarray('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.'), 32):
    print(magma_sub(block))

bitarray('11110010001010100010100110110000')
bitarray('11100100111111010000101100110100')
bitarray('11100001100001010000111011000100')
bitarray('11100100110111010000110000100100')
bitarray('10011010101011010000110001110010')
bitarray('11100001111111010000110000100100')
bitarray('11100010001011011001110000000010')
bitarray('10111010001011011111000110110000')
bitarray('11100010001010100010100110110000')
bitarray('10010010101000010000100110110011')
bitarray('10010010101011010000100110110000')
bitarray('11100001111111010000111011100100')
bitarray('11100100100111010000110001010100')
bitarray('11100100110110010000110000001100')
bitarray('11000010001010100010000110110011')
bitarray('10101010001011011110100110110000')
bitarray('11100100110101010000101100110100')
bitarray('11100100101011010000110001101100')
bitarray('11100001111110010101110001010100')
